In [8]:
from utils import validate
import utils
import random
import gymnasium as gym
from model import ImitationCNNNet
from model import PolicyNetwork
import torch
n_actions = 9
device = "cpu"
eval_model = ImitationCNNNet(n_actions).to(device)
eval_model.load_state_dict(torch.load('saved_model/best_model_2023-04-30.pth'))
# validate(eval_model, args.render, args.nepisodes)
validate(eval_model, False, 1)

render =  False
nepisodes =  1


[INFO:2023-04-30 19:16:08,478] Steps taken over each of 1 episodes: 783
[INFO:2023-04-30 19:16:08,479] Total return after 1 episodes: 1770.000


In [9]:
def run_evaluation(model):
    # We create the environment so it's new and read the observations saved into obs
    env = gym.make("ALE/MsPacman-v5")
    env_seed = random.randint(0, 9999)
    obs = env.reset(seed=env_seed)[0]
    obs = utils.preprocess_observation(obs).unsqueeze(0).to(device)
    # print("obs = ", obs.shape)
    total_reward = 0.0
    learn_pairs = {}
    time_stamp = 0
    done = False
    prev_state = None
    while not done:
        action, prev_state = model.get_action(obs, prev_state)
        next_obs, reward, done, another_done, info = env.step(action)
        total_reward += reward
        # print("action = ", action, " reward = ", reward, " done = ", done, " another_done = ", another_done, " info = ", info)
        learn_pairs[time_stamp] = (action, reward, total_reward)
        time_stamp += 1
        obs = utils.preprocess_observation(next_obs).unsqueeze(0).to(device)
    # print("you died!")
    env.close()
    return total_reward

In [11]:
eval_model = ImitationCNNNet(n_actions).to(device)
eval_model.load_state_dict(torch.load('saved_model/best_model_2023-04-30.pth'))
eval_model.eval()
save_path = "saved_model"
running_avg = 0.0
epr_num = 20
for idx in range(epr_num+1):
    new_reward = run_evaluation(eval_model)
    running_avg += (new_reward - running_avg) / (idx+1)
    running_avg = round(running_avg, 2)
    if idx % 20 == 0:
        print(f"Until Evaluation # {idx} Average reward = ", running_avg)

Until Evaluation # 0 Average reward =  1640.0
Until Evaluation # 20 Average reward =  1670.96
Until Evaluation # 40 Average reward =  1637.32
Until Evaluation # 60 Average reward =  1607.22
Until Evaluation # 80 Average reward =  1606.08
Until Evaluation # 100 Average reward =  1628.46
Until Evaluation # 120 Average reward =  1635.73
Until Evaluation # 140 Average reward =  1620.01
Until Evaluation # 160 Average reward =  1638.27
Until Evaluation # 180 Average reward =  1639.02
Until Evaluation # 200 Average reward =  1652.35
